Dependences to install

In [13]:
%pip install tensorflow==2.3.1
%pip install gym
%pip install keras-rl2
%pip install gym[atari]

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow==2.3.1 (from versions: 2.5.0rc0, 2.5.0rc1, 2.5.0rc2, 2.5.0rc3, 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.9.0rc0)
ERROR: No matching distribution found for tensorflow==2.3.1


  Using cached gym-0.23.1-py3-none-any.whl
  Using cached gym_notices-0.0.6-py3-none-any.whl (2.7 kB)
Note: you may need to restart the kernel to use updated packages.

  Using cached ale_py-0.7.4-cp39-cp39-win_amd64.whl (904 kB)



Environment : OpenAI Gym

In [1]:
import random
import gym
gym.__file__


'C:\\Users\\AbdelfataH\\anaconda3\\envs\\gym\\lib\\site-packages\\gym\\__init__.py'

In [2]:
env = gym.make('SpaceInvaders-v0')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [3]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [6]:
episodes = 5
for episode in range(1, episodes + 1):
    state = env.reset()
    done = False
    scoure = 0
    while not done:
        env.render()
        action = random.choice([0, 1, 2, 3, 4, 5])
        n_state, reward, done, info = env.step(action)
        scoure += reward
    print('Episode: {} Scoure: {}'.format(episode, scoure))
env.close()

Episode: 1 Scoure: 120.0
Episode: 2 Scoure: 65.0
Episode: 3 Scoure: 135.0
Episode: 4 Scoure: 330.0
Episode: 5 Scoure: 110.0


Deep Learning Model : Keras

In [2]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [4]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8, 8), strides=(4, 4), activation='relu', input_shape=(3, height, width, channels)))
    model.add(Convolution2D(64, (4, 4), strides=(2, 2), activation='relu'))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [7]:
model = build_model(height, width, channels, actions)

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten (Flatten)            (None, 67584)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               34603520  
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 1

In [ ]:
del model

Build Agent : Keras-RL

In [3]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [5]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit = 1000, window_length = 3)
    dqn = DQNAgent(model = model, memory = memory, policy = policy, 
                    enable_dueling_network=True, dueling_type='avg',
                    nb_actions = actions, nb_steps_warmup = 1000)
    return dqn

In [ ]:
model = build_model(height, width, channels, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-4))
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

Reloading Agent from Memory

In [ ]:
dqn.save_weights('10k/Agent02.h5f')

In [ ]:
del model, dqn

In [6]:

env = gym.make('SpaceInvaders-v0')
height, width, channels = env.observation_space.shape
actions = env.action_space.n
model = build_model(height, width, channels, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-4))

In [7]:
dqn.load_weights('1m/dqn_weights.h5f')

In [8]:
scoures = dqn.test(env, nb_episodes = 10, visualize = True)
print(np.mean(scoures.history['episode_reward']))

Testing for 10 episodes ...


C:\Users\AbdelfataH\anaconda3\envs\gym\lib\site-packages\keras\engine\training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
C:\Users\AbdelfataH\anaconda3\envs\gym\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] Impossible de modifier le mode thread une fois qu’il a été fixé
  warnings.warn(str(err))


Episode 1: reward: 165.000, steps: 636
Episode 2: reward: 205.000, steps: 994
Episode 3: reward: 100.000, steps: 547
Episode 4: reward: 260.000, steps: 1215
Episode 5: reward: 495.000, steps: 1682
Episode 6: reward: 60.000, steps: 399
Episode 7: reward: 270.000, steps: 961
Episode 8: reward: 455.000, steps: 868
Episode 9: reward: 330.000, steps: 1168
Episode 10: reward: 300.000, steps: 899
264.0


In [9]:
env.close()